In [58]:
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ACTIVELOOP_TOKEN = os.getenv("ACTIVELOOP_TOKEN")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")



In [59]:
print(PINECONE_API_KEY)

pcsk_5cnXhG_N7NZjKLpsFx4xcb5Xc68jE7wUe3xSUHzVykkt4wXTRMhmePERUzNoF7968qED27


In [2]:
import yt_dlp
import warnings
warnings.filterwarnings("ignore")

In [3]:
def download_mp4_from_youtube(url):
    # set the details for the download
    filename = "llm_video.mp4"
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',  # Fixed format string syntax
        'outtmpl': filename,
        'quiet': True,
        'nocheckcertificate': True  # Add this to bypass SSL verification
    }

    # download the video
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(url, download=True)

In [13]:
url = "https://www.youtube.com/watch?v=zjkBMFhNj_g"
download_mp4_from_youtube(url)

In [4]:
# Using whisper to transcribe the video

import whisper

model = whisper.load_model("base")
result = model.transcribe("llm_video.mp4.webm")
print(result["text"])

 Hi everyone. So recently I gave a 30 minute talk on large language models just kind of like an intro talk Um, unfortunately that talk was not recorded But a lot of people came to me after the talk and they told me that they really liked the talk So I was just I thought I was just re-recorded and basically put it up on YouTube So here we go the busy persons intro to large language models director scut Okay, so let's begin First of all, what is a large language model really? Well a large language model is just two files, right? Um, there will be two files in this hypothetical directory So for example, where can we do a specific example of the llama to 70b model? This is a large language model released by meta AI And this is basically the llama series of language models the second iteration of it and this is the 70 billion parameter model of Of this series. So there's multiple models belong to the llama to series 7 billion 13 billion 34 billion and 70 billion as the biggest one Now many 

In [5]:
with open("transcript.txt", "w") as file:
    file.write(result["text"])

## Summarization using LangChain

In [7]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain

In [8]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", 
             temperature=0)

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=0,
    separators = [" ",",","\n"]
    )

In [10]:
from langchain.docstore.document import Document
with open("transcript.txt") as file:
    text = file.read()

In [11]:
# Split the transcript text into smaller chunks using the text splitter
texts = text_splitter.split_text(text)

# Convert all text chunks into Document objects
docs = [Document(page_content=t) for t in texts]

In [12]:
print(docs)

[Document(metadata={}, page_content="Hi everyone. So recently I gave a 30 minute talk on large language models just kind of like an intro talk Um, unfortunately that talk was not recorded But a lot of people came to me after the talk and they told me that they really liked the talk So I was just I thought I was just re-recorded and basically put it up on YouTube So here we go the busy persons intro to large language models director scut Okay, so let's begin First of all, what is a large language model really? Well a large language model is just two files, right? Um, there will be two files in this hypothetical directory So for example, where can we do a specific example of the llama to 70b model? This is a large language model released by meta AI And this is basically the llama series of language models the second iteration of it and this is the 70 billion parameter model of Of this series. So there's multiple models belong to the llama to series 7 billion 13 billion 34 billion and 70 

### Summarization using "Stuff" method - using create_stuff_documents_chain

In [15]:
# Import required modules from LangChain
from langchain.chains.combine_documents import create_stuff_documents_chain  # For creating a chain that combines documents using the "stuff" method
from langchain_core.prompts import ChatPromptTemplate  # For creating chat-style prompts
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# Define the prompt template that will be used for summarization
# ChatPromptTemplate.from_messages creates a prompt from a list of (role, content) tuples
# Here we use a "system" role with instructions to write a concise summary
# {context} is a placeholder that will be filled with our document content
prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a concise summary of the following:\\n\\n{context}")]
)

# Create a chain that will:
# 1. Take our documents (docs)
# 2. "Stuff" them all into the prompt's {context}
# 3. Send to the LLM (defined earlier as 'llm')
# 4. Get back a summary
chain = create_stuff_documents_chain(llm, prompt)

# Execute the chain by:
# 1. Passing our documents in a dictionary with "context" as the key
# 2. The chain will process them according to the steps above
# 3. Store the summary result in 'result'
result = chain.invoke({"context": docs})
print(result)  # Print the generated summary

The speaker recently delivered a 30-minute introductory talk on large language models (LLMs), specifically focusing on the Llama 2 70B model released by Meta AI. LLMs consist of two main files: a parameters file containing the model's weights and a run file that executes the model. The Llama 2 70B model is notable for being one of the most powerful open-weight models available, allowing users to run it independently without internet connectivity.

Training an LLM involves processing vast amounts of text data (around 10 terabytes) using a GPU cluster over several days, costing millions of dollars. The model learns to predict the next word in a sequence, effectively compressing knowledge from the internet into its parameters. The speaker explains the difference between pre-training (learning from internet documents) and fine-tuning (training on high-quality Q&A datasets) to create assistant models capable of answering questions.

The talk also highlights the evolution of LLMs, including 

In [16]:
import textwrap

wrapped_text = textwrap.fill(result, width=100)
print(wrapped_text)


The speaker recently delivered a 30-minute introductory talk on large language models (LLMs),
specifically focusing on the Llama 2 70B model released by Meta AI. LLMs consist of two main files:
a parameters file containing the model's weights and a run file that executes the model. The Llama 2
70B model is notable for being one of the most powerful open-weight models available, allowing users
to run it independently without internet connectivity.  Training an LLM involves processing vast
amounts of text data (around 10 terabytes) using a GPU cluster over several days, costing millions
of dollars. The model learns to predict the next word in a sequence, effectively compressing
knowledge from the internet into its parameters. The speaker explains the difference between pre-
training (learning from internet documents) and fine-tuning (training on high-quality Q&A datasets)
to create assistant models capable of answering questions.  The talk also highlights the evolution
of LLMs, including

In [17]:
from langchain.prompts import PromptTemplate

prompt_template = """Write a concise bullet point summary of the following:

{context}

CONSCISE SUMMARY IN BULLET POINTS:"""

bullet_point_prompt = PromptTemplate(template=prompt_template, 
                        input_variables=["context"])

chain = create_stuff_documents_chain(llm, bullet_point_prompt)

result = chain.invoke({"context": docs})
print(result)  # Print the generated summary


- **Talk Overview**: The speaker presented an introductory talk on large language models (LLMs) that was well-received but not recorded, prompting a re-recording for YouTube.

- **Definition of LLMs**: LLMs consist of two main files: a parameters file (weights) and a code file to run the model, exemplified by the Llama 2 70B model from Meta AI.

- **Model Characteristics**: 
  - Llama 2 70B is an open-source model with 70 billion parameters, making it one of the most powerful available.
  - The parameters file is approximately 140 GB, and the model can be run locally without internet connectivity.

- **Training Process**: 
  - Model training involves compressing vast amounts of internet text (around 10 terabytes) using a GPU cluster (6000 GPUs over 12 days, costing about $2 million).
  - The training process is computationally intensive, while inference (running the model) is relatively cheap.

- **Next Word Prediction**: LLMs predict the next word in a sequence, learning extensive kno

In [18]:
wrapped_text = textwrap.fill(result, 
                             width=1000,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

- **Talk Overview**: The speaker presented an introductory talk on large language models (LLMs) that was well-received but not recorded, prompting a re-recording for YouTube.

- **Definition of LLMs**: LLMs consist of two main files: a parameters file (weights) and a code file to run the model, exemplified by the Llama 2 70B model from Meta AI.

- **Model Characteristics**: 
  - Llama 2 70B is an open-source model with 70 billion parameters, making it one of the most powerful available.
  - The parameters file is approximately 140 GB, and the model can be run locally without internet connectivity.

- **Training Process**: 
  - Model training involves compressing vast amounts of internet text (around 10 terabytes) using a GPU cluster (6000 GPUs over 12 days, costing about $2 million).
  - The training process is computationally intensive, while inference (running the model) is relatively cheap.

- **Next Word Prediction**: LLMs predict the next word in a sequence, learning extensive
kno

## Adding Transcripts to Deep Lake

When we have a large number of transcripts, we can store them in a Deep Lake database.

In [19]:
import yt_dlp

In [20]:
# Function to download multiple YouTube videos as MP4 files
def download_mp4_from_youtube(urls, job_id):
    # Initialize empty list to store video information
    video_info = []

    # Loop through each URL with an index counter
    for i, url in enumerate(urls):
        # Create unique filename using job_id and index
        file_temp = f'./{job_id}_{i}.mp4'
        
        # Configure youtube-dl options:
        # - Download best quality MP4 video and audio
        # - Save with the temporary filename
        # - Run quietly without progress output
        ydl_opts = {
            'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
            'outtmpl': file_temp,
            'quiet': True
        }
        
        # Download the video using youtube-dl
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            # Extract video info and download
            result = ydl.extract_info(url, download=True)
            # Get video title and author, default to empty string if not found
            title = result.get('title', "")
            author = result.get('uploader', "")
        
        # Store filename, title and author for each video
        video_info.append((file_temp, title, author))
    
    # Return list of tuples containing info for all downloaded videos
    return video_info   

# List of YouTube video URLs to download
urls = ["https://www.youtube.com/watch?v=2IK3DFHRFfw&t=616s", "https://www.youtube.com/watch?v=fkIvmfqX-t0", "https://www.youtube.com/watch?v=KrRD7r7y7NY"]

# Download the videos and get their details
video_details = download_mp4_from_youtube(urls, 1)

Now, transcribe the videos using Whisper as we previously saw and save the results in a text file.

In [21]:
import whisper

# load the model
model = whisper.load_model("base")

# iterate through each video and transcribe
results = []
for video in video_details:
    result = model.transcribe(video[0])
    results.append( result['text'] )
    print(f"Transcription for {video[0]}:\n{result['text']}\n")

Python(94629) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Transcription for ./1_0.mp4:
 Every since computers were invented, they've really just been glorified calculators. Machines that execute the exact instructions given to them by the programmers. But something incredible is happening now. Computers have started gaining the ability to learn and think and communicate, just like we do. They can do creative intellectual work that previously only humans could do. We call this technology generative AI, and you may have encountered it already through products like chat GPT. Basically, intelligence is now available as a service, kind of like a giant brain floating in the sky that anyone can talk to. It's not perfect, but it is surprisingly capable and it is improving at an exponential rate. This is a big deal. It's going to affect just about every person and company on the planet, positively or negatively. This video is here to help you understand what generative AI is all about, in practical terms, beyond the hype. The better you understand thi

Python(94640) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Transcription for ./1_1.mp4:
 Hello world, my name is David J. Maillen and I'm a professor of computer science at Harvard University. Today I've been asked to explain algorithms in five levels of increasing difficulty. Algorithms are important because they really are everywhere, not only in the physical world but certainly in the virtual world as well. And in fact what excites me about algorithms is that they really represent an opportunity to solve problems. And I dare say no matter what you do in life, all of us have problems to solve. So I'm a computer science professor so I spend a lot of time with computers. How would you define a computer for them? Well a computer is electronic, like a phone but it's a rectangle and you like kid type, like, and you work on it. Nice. Do you know any of the parts that are inside of a computer? No. Can I explain a couple of them to you? Yeah. So like inside of every computer is some kind of brain and the technical term for that is CPU or central pro

Python(94650) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Transcription for ./1_2.mp4:
 Please welcome Andrew Aing. Thank you. It's such a good time to be a build. There are some insights to be back here at Snowflake Build. What I'd like to do today is share of you where I think are some of AI's biggest opportunities. You may have heard me say that I think AI is a new electricity. That's because AI is a general purpose technology like electricity. I've asked you what is electricity good for is also hard to answer because it's good for so many different things. And new AI technology is creating a few set of opportunities for the build new applications that weren't possible before. People often ask me, hey Andrew, where are the biggest AI opportunities? This is what I think of as AI stack. At the lowest level is the semiconductor. And then on top of that, lot of the cloud in frow twos, including of course, Snowflake. And then on top of that are many of the foundation models, trainers and models. And it turns out that a lot of the meteor hype an

In [22]:
with open ('text.txt', 'w') as file:  
    for result in results:  # results is a list, so iterate through it
        file.write(result + '\n')  # write each transcription 

In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create text splitter first
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
    )

# Load and split the texts
with open('text.txt') as f:
    text = f.read()
texts = text_splitter.split_text(text)

In [41]:
from langchain.docstore.document import Document

# Convert all text chunks into Document objects
docs = [Document(page_content=t) for t in texts]

In [60]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)



In [63]:
import time

index_name = "youtube-summarizer"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [64]:

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [67]:
from uuid import uuid4
from langchain_pinecone import PineconeVectorStore
uuids = [str(uuid4()) for _ in range(len(docs))]

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

vector_store.add_documents(documents=docs, ids=uuids)

['47c49ec6-d0a1-4e99-bbab-9cceacf13913',
 '5608c003-e0e6-4243-b393-9145f379c4b6',
 '34d04bb6-00f6-44a4-8090-7431f512c323',
 '78ca1f71-b612-4bdf-8661-534dcdda6a20',
 '42391365-f0a5-4b9b-b8c0-a843f62fb395',
 'ac8eabb9-5467-48fc-9f87-4fcbc2c59e3e',
 'a5be021f-8d59-4e18-b65b-923348f5d92a',
 '82dffa35-7f0c-4692-82fe-6e80ea6089fb',
 '136968e5-c6d8-409d-a4b6-be878865b5d8',
 '59a65541-c58b-4ee3-9ac4-d520a1526cda',
 '9a8540db-2a5e-4390-9136-67fb9962636f',
 '7c7dcd90-0566-41a5-a1d9-81a7953754b1',
 '9284bad7-83c6-4cb2-8535-ff081e52d84a',
 '20fa41e4-6778-4323-9a06-247a46d1bac8',
 'a1388e1d-7874-4af6-ad6a-c9b3bd441c95',
 'd01c7bce-3c55-412c-a3e0-0fe06048ef17',
 'def2874c-d1b4-4fea-ad8d-0cde7ea43964',
 'cc545234-a183-49d2-92a4-58414415d4bb',
 'b6655ee2-9071-4aaf-a0ca-8316192b5800',
 'c71c1b7e-af67-4cd2-8cac-99ad5a19c0d4',
 'f48face9-c7ec-40a3-b9a9-d9457934bc39',
 '6e79e743-8aa7-4ade-9ca5-32f34aa9e74f',
 '5eaa758e-de87-41b2-9cd1-22fd207a1e6f',
 'dd22aa67-b438-474a-8b28-2871307d53f4',
 'f2567804-9f05-

In [72]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 5, "score_threshold": 0.5},
)

In [73]:
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of transcripts from a video to answer the question in bullet points and summarized. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Summarized answer in bullter points:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [75]:
from langchain.chains import RetrievalQA
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 chain_type_kwargs=chain_type_kwargs)

print( qa.run("WHat is teh video about?") )

- The video demonstrates a demo app that utilizes a vision agent to index and search video content based on metadata.
- It showcases how users can search for specific actions (e.g., "skier airborne") and view clips with high similarity highlighted on a timeline.
- The speaker emphasizes the potential of generative AI as a powerful tool for individuals and organizations, encouraging viewers to understand and leverage it.
- Key points include the importance of prompt engineering skills and the need for experimentation to improve proficiency in using generative AI.
- The video discusses the emergence of multi-modal AI products that integrate various types of media (text, images, audio) into a single tool.
- The speaker shares personal experiences using AI for brainstorming and problem-solving.
- Generative AI is described as a transformative technology that enables computers to perform creative and intellectual tasks, akin to human capabilities.
- The video aims to provide practical insig